In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with TEASER-EASE model and simulated feedback

In [2]:
from pathlib import Path
import numpy as np

import teaser.util as util
import teaser.evaluation as evaluation

from teaser.algorithm.teaser_ease import TEASER_EASE

## Datasets

In [3]:
## Uncomment corresponding directory.
DATA_DIR = Path('../../data/ML20M/')
# DATA_DIR = Path('../../data/Amazon Video Games/')
# DATA_DIR = Path('../../data/Publiq/')

In [4]:
# Shouldn't need to change this info
INTERACTIONS_TRAIN_PATH = DATA_DIR / 'interactions.train.csv'
INTERACTIONS_VALIN_PATH = DATA_DIR / 'interactions.valin.csv'
INTERACTIONS_VALOUT_PATH = DATA_DIR / 'interactions.valout.csv'
INTERACTIONS_TESTIN_PATH = DATA_DIR / 'interactions.testin.csv'
INTERACTIONS_TESTOUT_PATH = DATA_DIR / 'interactions.testout.csv'

METADATA_PATH = DATA_DIR / 'features.csv'

ITEM_ID = 'itemId'
USER_ID = 'userId'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = TEASER_EASE

# ML20M
L2_EASE = 500
L2_1 = 100
L2_2 = 100000000
RHO = 100
DELTA = 0
MAX_ITER = 10

# AVG
# L2_EASE = 100
# L2_1 = 500
# L2_2 = 2000
# RHO = 10000
# DELTA = 0
# MAX_ITER = 15

# Publiq
# L2_EASE = 10
# L2_1 = 500
# L2_2 = 50000
# RHO = 1000
# DELTA = 0
# MAX_ITER = 10

HYPERPARAMS = {'l2_ease': L2_EASE, 'l2_1': L2_1, 'l2_2': L2_2, 'rho': RHO, 'delta': DELTA,'max_iterations': MAX_ITER}
HYPERPARAMS

{'l2_ease': 500,
 'l2_1': 100,
 'l2_2': 100000000,
 'rho': 100,
 'delta': 0,
 'max_iterations': 10}

## Parse data

In [6]:
S, tags = util.parse_metadata(METADATA_PATH, item_id=ITEM_ID)
n, t = S.shape  # amount of items and tags

X = util.parse_interactions(INTERACTIONS_TRAIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

Xtest_in = util.parse_interactions(INTERACTIONS_TESTIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)
Xtest_out = util.parse_interactions(INTERACTIONS_TESTOUT_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

In [7]:
# # debug
# d_items = 1000
# S = S[:d_items,:]
# tags = tags[:d_items]
# X = X[:,:d_items]
# Xtest_in = Xtest_in[:,:d_items]
# Xtest_out = Xtest_out[:,:d_items]

## Evaluate model with optimal hyperparams on test set

In [8]:
%%time
alg = ALG(**HYPERPARAMS)
alg.fit(X, S)

Decompose XTX
Decompose DTD


norm E 0.044672216207076614
diag norm: 0.19702029516317016
diag_diff: 0.19702029516317016

norm E 0.044671703830224764
diag norm: 0.19700926989788112
diag_diff: 0.19700926989788112
rho * change E: 0.00014114594746936798

norm E 0.044671191520337314
diag norm: 0.19699824549789988
diag_diff: 0.19699824549789988
rho * change E: 0.0001411365387456688

norm E 0.04467067927740572
diag norm: 0.19698722196313428
diag_diff: 0.19698722196313428
rho * change E: 0.00014112713089108923

norm E 0.044670167101421576
diag norm: 0.19697619929349164
diag_diff: 0.19697619929349164
rho * change E: 0.00014111772390642112

norm E 0.04466965499237638
diag norm: 0.19696517748887976
diag_diff: 0.19696517748887976
rho * change E: 0.00014110831779057134

norm E 0.044669142950261545
diag norm: 0.19695415654920614
diag_diff: 0.19695415654920614
rho * change E: 0.00014109891254413066

norm E 0.044668630975068714
diag norm: 0.19694313647437836
diag_diff: 0.19694313647437836
rho * change E: 0.00014108950816664315

no

In [9]:
print("Performance on test set with simulation")
for POS_FEEDBACK in [1, 2]:
    for STRENGTH in [3]:
        print("Pos feedback count:", POS_FEEDBACK, "with strength (out of 5):", STRENGTH)
        score = evaluation.simulate(alg, Xtest_in, Xtest_out, S, pos_feedback=POS_FEEDBACK, strength=STRENGTH, repeats=3)
    print()

Performance on test set with simulation
Pos feedback count: 1 with strength (out of 5): 3
Evaluating with 10000 users
Average Recall@20 0.365
Average Recall@100 0.61
Average nDCG@100 0.394

Pos feedback count: 2 with strength (out of 5): 3
Evaluating with 10000 users
Average Recall@20 0.373
Average Recall@100 0.617
Average nDCG@100 0.4

